In [1]:
import pandas as pd
df=pd.read_csv(r"Pluviales\201301010000Lluv.csv", encoding="latin-1")

In [2]:
df.columns

Index(['LON', '   LAT', '   ESTADO', '     CVE_SIH', 'NOMBRE_ESTACION', 'ENE'], dtype='object')

In [12]:
import pandas as pd
import glob
import os

# Configuración de ruta
ruta_busqueda = r"Pluviales\*.csv"
archivos = glob.glob(ruta_busqueda)

# Verificación de existencia de archivos
if not archivos:
    raise FileNotFoundError("No se encontraron archivos en la ruta especificada.")

# Establecer referencia (Primer archivo)
archivo_base = archivos[0]
cols_referencia = list(pd.read_csv(archivo_base, encoding="latin-1", nrows=0).columns)

print(f"Archivo base: {os.path.basename(archivo_base)}")
print(f"Columnas referencia: {cols_referencia}")
print("-" * 40)

archivos_diferentes = {}

# Iteración y comparación
for archivo in archivos[1:]:
    try:
        # Leer solo encabezados
        cols_actual = list(pd.read_csv(archivo, encoding="latin-1", nrows=0).columns)
        
        # Comparar longitud y nombres exactos
        if cols_actual != cols_referencia:
            # Identificar diferencias de conjuntos
            set_ref = set(cols_referencia)
            set_act = set(cols_actual)
            faltantes = set_ref - set_act
            sobrantes = set_act - set_ref
            
            archivos_diferentes[os.path.basename(archivo)] = {
                "cols_actuales": cols_actual,
                "faltantes_en_archivo": list(faltantes),
                "sobrantes_en_archivo": list(sobrantes)
            }
            
    except Exception as e:
        archivos_diferentes[os.path.basename(archivo)] = f"Error de lectura: {str(e)}"

# Reporte de resultados
if not archivos_diferentes:
    print("ÉXITO: Todos los archivos comparten exactamente las mismas columnas.")
else:
    print(f"ADVERTENCIA: Se encontraron {len(archivos_diferentes)} archivos con inconsistencias:\n")
    for nombre, detalle in archivos_diferentes.items():
        print(f"Archivo: {nombre}")
        print(f"Detalle: {detalle}")
        print("-" * 20)

Archivo base: 201301010000Lluv.csv
Columnas referencia: ['LON', '   LAT', '   ESTADO', '     CVE_SIH', 'NOMBRE_ESTACION', 'ENE']
----------------------------------------
ADVERTENCIA: Se encontraron 143 archivos con inconsistencias:

Archivo: 201302010000Lluv.csv
Detalle: {'cols_actuales': ['  LON', '   LAT', '   ESTADO', '     CVE_SIH', 'NOMBRE_ESTACION', ' FEB'], 'faltantes_en_archivo': ['ENE', 'LON'], 'sobrantes_en_archivo': ['  LON', ' FEB']}
--------------------
Archivo: 201303010000Lluv.csv
Detalle: {'cols_actuales': ['  LON', '   LAT', '   ESTADO', '     CVE_SIH', 'NOMBRE_ESTACION', ' MAR'], 'faltantes_en_archivo': ['ENE', 'LON'], 'sobrantes_en_archivo': ['  LON', ' MAR']}
--------------------
Archivo: 201304010000Lluv.csv
Detalle: {'cols_actuales': ['  LON', '   LAT', '   ESTADO', '     CVE_SIH', 'NOMBRE_ESTACION', ' ABR'], 'faltantes_en_archivo': ['ENE', 'LON'], 'sobrantes_en_archivo': ['  LON', ' ABR']}
--------------------
Archivo: 201305010000Lluv.csv
Detalle: {'cols_actuale

In [13]:
import pandas as pd
import glob
import os

# Configuración
ruta_archivos = r"Pluviales\*.csv"
archivos = glob.glob(ruta_archivos)

dfs = []

# Mapeo de columnas estándar (Normalizado a mayúsculas y sin espacios)
# Estructura: {Nombre_Final: [Lista de posibles nombres originales]}
mapa_columnas = {
    'LON': ['LON', 'XLO', 'LONGITUD'],
    'LAT': ['LAT', 'YLA', 'LATITUD'],
    'ESTADO': ['ESTADO', 'EDO'],
    'CLAVE': ['CLAVE', 'CVE_SIH', 'CLAVE_SIH', 'CVE', 'EST'], # 'EST' para caso 202304
    'NOMBRE': ['NOMBRE', 'NOMBRE_ESTACION', 'ESTACION', 'NAME']
}

print(f"Procesando {len(archivos)} archivos...")

for archivo in archivos:
    try:
        # 1. Extracción de metadatos del nombre del archivo
        nombre_archivo = os.path.basename(archivo)
        anio = nombre_archivo[:4]
        mes = nombre_archivo[4:6]
        
        # 2. Lectura del archivo
        # Se asume encoding latin-1 por los acentos en México
        df = pd.read_csv(archivo, encoding="latin-1")
        
        # 3. Limpieza de encabezados
        # Eliminar espacios, convertir a mayúsculas
        df.columns = df.columns.str.strip().str.upper()
        
        # Eliminar columnas 'Unnamed' (basura generada por comas extra)
        cols_validas = [c for c in df.columns if not c.startswith('UNNAMED')]
        df = df[cols_validas]
        
        # 4. Normalización de columnas
        cols_finales = {}
        col_precipitacion = None
        
        columnas_identificadas = []
        
        for col in df.columns:
            identificada = False
            for estandar, variantes in mapa_columnas.items():
                if col in variantes:
                    cols_finales[col] = estandar
                    columnas_identificadas.append(estandar)
                    identificada = True
                    break
            
            # Si no es una columna estándar de ubicación/id, asumimos que es la precipitación
            if not identificada:
                if col_precipitacion is None:
                    col_precipitacion = col
                    cols_finales[col] = 'PRECIPITACION'
                else:
                    # Si ya tenemos precipitación y aparece otra columna desconocida, es una anomalía
                    # Pero en este dataset, asumiremos que la primera desconocida es la lluvia
                    pass

        # Renombrar
        df = df.rename(columns=cols_finales)
        
        # 5. Estandarización final
        # Asegurar que existan todas las columnas base, si falta alguna se rellena con NaN
        columnas_base = ['CLAVE', 'NOMBRE', 'ESTADO', 'LON', 'LAT', 'PRECIPITACION']
        for col in columnas_base:
            if col not in df.columns:
                df[col] = None 
                
        # Agregar columnas temporales
        df['ANIO'] = anio
        df['MES'] = mes
        
        # Seleccionar y ordenar
        df_final = df[columnas_base + ['ANIO', 'MES']]
        
        dfs.append(df_final)

    except Exception as e:
        print(f"Error procesando {nombre_archivo}: {e}")

# Consolidación
if dfs:
    df_consolidado = pd.concat(dfs, ignore_index=True)
    
    # Conversión de tipos para optimizar y limpiar
    # Convertir precipitación a numérico, forzando errores a NaN (ej. trazas o textos)
    df_consolidado['PRECIPITACION'] = pd.to_numeric(df_consolidado['PRECIPITACION'], errors='coerce')
    
    print("-" * 30)
    print(f"Consolidación completada.")
    print(f"Total registros: {len(df_consolidado)}")
    display(df_consolidado.head())
else:
    print("No se generaron datos para consolidar.")

Procesando 144 archivos...
------------------------------
Consolidación completada.
Total registros: 160080


,CLAVE,NOMBRE,ESTADO,LON,LAT,PRECIPITACION,ANIO,MES
0,AGSAG,AGUASCALIENTES,AGS,-102.31,21.90,58.5,2013,01
1,ANVAG,CINCUENTA ANIVERSARIO,AGS,-102.47,22.19,0.0,2013,01
2,BRTAG,SAN BARTOLO,AGS,-102.18,21.74,17.0,2013,01
3,CDRAG,LA CODORNIZ,AGS,-102.67,22.00,45.0,2013,01
4,CLVAG,CALVILLITO,AGS,-102.19,21.83,60.5,2013,01


In [15]:
# Asumiendo que 'df_consolidado' está en memoria del paso anterior.
# Si no, cargar: df_consolidado = pd.read_csv("Consolidado_Pluviales.csv")

# 1. Normalización de la columna ESTADO
# Se convierte a mayúsculas y se eliminan espacios al inicio/final
df_consolidado['ESTADO'] = df_consolidado['ESTADO'].astype(str).str.strip().str.upper()

# 2. Definición de criterios de búsqueda
# Incluye nomenclatura antigua (DF) y actual (CDMX)
variantes_df = 'DF'

# 3. Filtrado
df_df = df_consolidado[df_consolidado['ESTADO'] == variantes_df].copy()

# 4. Verificación y Exportación
print(f"Registros encontrados para DF/CDMX: {len(df_df)}")
print("Variantes encontradas en la columna ESTADO:", df_df['ESTADO'].unique())

if not df_df.empty:
    display(df_df.head())
else:
    print("No se encontraron registros que coincidan con DF/CDMX.")

Registros encontrados para DF/CDMX: 9672
Variantes encontradas en la columna ESTADO: ['DF']


,CLAVE,NOMBRE,ESTADO,LON,LAT,PRECIPITACION,ANIO,MES
189,AJSDF,AJUSCO,DF,-99.21,19.22,20.5,2013,01
190,CBIDF,"ESCUELA NACIONAL DE CIENCIAS BIOLOGICAS, IPN (...",DF,-99.17,19.45,1.3,2013,01
191,CCUDF,CENTRO UNIVERSITARIO,DF,-99.18,19.35,0.2,2013,01
192,CDMDF,CARCEL DE MUJERES,DF,-99.00,19.36,0.0,2013,01
193,CDVDF,CIUDAD DEPORTIVA,DF,-99.11,19.41,0.8,2013,01


In [17]:
import os

# Configuración de ruta
directorio_salida = "Data_Tratado"
nombre_archivo = "Pluviales_DF.csv"

# Crear directorio (exist_ok=True evita error si ya existe)
os.makedirs(directorio_salida, exist_ok=True)

# Construcción de ruta absoluta/relativa
ruta_completa = os.path.join(directorio_salida, nombre_archivo)

# Exportar
try:
    df_df.to_csv(ruta_completa, index=False, encoding='utf-8-sig')
    print(f"EXPORTACIÓN EXITOSA: {ruta_completa}")
except NameError:
    print("ERROR: El DataFrame 'df_df' no está definido en memoria. Ejecute el filtrado previo.")
except Exception as e:
    print(f"ERROR DE ESCRITURA: {str(e)}")

EXPORTACIÓN EXITOSA: Data_Tratado\Pluviales_DF.csv


In [ ]:
df_df.to_csv(ruta_completa, index=False, encoding='utf-8-sig')

In [19]:
df_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9672 entries, 189 to 159307
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CLAVE          9672 non-null   object 
 1   NOMBRE         9672 non-null   object 
 2   ESTADO         9672 non-null   object 
 3   LON            9672 non-null   float64
 4   LAT            9672 non-null   float64
 5   PRECIPITACION  9672 non-null   float64
 6   ANIO           9672 non-null   object 
 7   MES            9672 non-null   object 
dtypes: float64(3), object(5)
memory usage: 680.1+ KB


In [20]:
df_df.describe()

,LON,LAT,PRECIPITACION
count,9672.000000,9672.000000,9672.000000
mean,-99.147791,19.361229,57.033590
std,0.082678,0.096949,71.495107
min,-99.440556,18.861389,0.000000
25%,-99.205000,19.300000,2.800000
50%,-99.142500,19.360000,24.600000
75%,-99.090000,19.435556,92.932500
max,-98.960000,19.633333,629.250000
